In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import min_max_norm

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data = pd.read_csv('D:/ProFile/Latent/biogeme/optima_keras.csv', low_memory=False)
random_num = 500
S=tf.Session()

In [3]:
trainable = True

In [4]:
train_latent = train_data[['ContIncome_0_4000','ContIncome_10000_more', 'ContIncome_4000_6000', 
                           'ContIncome_6000_8000', 'ContIncome_8000_10000', 
                           'age_65_more','haveChildren', 
                           'haveGA', 'highEducation', 'individualHouse', 'male',
                           'moreThanOneBike', 'moreThanOneCar']]

train_PTcom = train_data[['MarginalCostPT_scaled', 'PurpHWH', 'PurpOther']]
train_CARcom = train_data[['CostCarCHF_scaled', 'PurpHWH', 'PurpOther']]

train_PTasc = train_data[['TimePT_scaled', 'WaitingTimePT']]
train_CARasc = train_data[['TimeCar_scaled']]
train_SMasc = train_data[['distance_km_scaled']]

train_indicator = train_data[['Envir01','Envir02','Envir03','Mobil11','Mobil14','Mobil16','Mobil17']]

train_y = train_data[['Choice']]

In [5]:
class LL(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LL, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = (train_indicator.shape[1], )
        # Create a trainable weight variable for this layer.
        self.sigma = self.add_weight(name='sigma',
                                      shape=shape,
                                     initializer=keras.initializers.Constant(value=np.ones(train_indicator.shape[1])),
#                                      initializer=keras.initializers.Constant(
#                                         value=np.array([1.25,1.12,1.07,1.08,1.05,1.1,1.11])),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(LL, self).build(input_shape)

    def call(self, inputs):
#         error = tf.subtract(inputs[0],inputs[1])
#         se=K.square(error)
        se=K.square(inputs)
        term1 = tf.divide(se, tf.multiply(2., K.square(self.sigma)))
        term2 = K.log(self.sigma)
        term3 = tf.multiply(0.5, K.log(6.283185))
        return tf.add(term1, tf.add(term2, term3))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(LL, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [6]:
class Subtract(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Subtract, self).__init__(**kwargs)

    def build(self, input_shape):

        # Be sure to call this at the end
        super(Subtract, self).build(input_shape)

    def call(self, inputs):
        return tf.cast(tf.subtract(inputs[0],inputs[1]),tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(Subtract, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [7]:
def LL_loss(y_true, y_pred):
    return K.sum(K.sum(K.abs(y_pred - y_true), axis=0))

In [8]:
def multi_indicator_loss(y_true, y_pred):
    n=train_data.shape[0]
    k1,k2 = 14,16
    error2 = K.sum(K.abs(y_pred - y_true), axis=0)
    term1 = tf.multiply(0.5, n-k2)
    term2_1 = tf.multiply(0.5, tf.log(tf.divide(error2, n-k2)))
    term2_2 = tf.multiply(0.5, tf.log(6.283185))
    term2 = tf.multiply(tf.cast(n, tf.float32), tf.add(term2_1, term2_2))
    L2 = tf.add(term1, term2)

    return K.sum(L2)

In [141]:
input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
y_ture = keras.Input(shape = (train_indicator.shape[1],), name = 'input_y_ture')
# car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear',
#                             kernel_initializer=keras.initializers.Constant(
#                                 value=np.array([[ -0.0265 ],
#                                                 [-0.0863],
#                                                 [0.2],
#                                                 [-0.205],
#                                                 [0.541],
#                                                 [-0.077],
#                                                 [0.0182],
#                                                 [ 0.724],
#                                                 [ 0.288],
#                                                 [ 0.0764],
#                                                 [-0.0694],
#                                                 [ 0.293],
#                                                 [-0.68]])),
#                                bias_initializer=keras.initializers.Constant(value=2.27))(input_latent)
car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear')(input_latent)

indicator_ref = keras.layers.Dense(1, name = 'indicator_ref',activation = 'linear',
                               kernel_initializer=keras.initializers.Constant(value=1),
                               bias_initializer=keras.initializers.Constant(value=0))(car_sys)
indicator_other = keras.layers.Dense(train_indicator.shape[1]-1, 
                                     name = 'indicator_other',
                                   kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=0., seed=None),
#                                      kernel_initializer=keras.initializers.Constant(
#                                          value=np.array([0.493,-0.706,-0.626,-0.707,-0.542,-0.456])),
#                                      bias_initializer=keras.initializers.Constant(
#                                          value=np.array([2.02,4.64,5.3,4.9,4.79,4.53,])),
                                     activation = 'linear')(car_sys)

indicator = keras.layers.Concatenate(axis=-1)([indicator_ref,indicator_other])

error = Subtract(1)([indicator, y_ture])

loglikelihood = LL(1)(error)

# idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
#                     outputs = indicator)

idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
                    outputs = loglikelihood)

#idmodel.layers[3].trainable = False
idmodel.layers[2].trainable = False
#idmodel.layers[7].trainable = False

idmodel.compile(optimizer=keras.optimizers.Adam(0.01),
              loss = LL_loss)

LLoutput = np.zeros((train_latent.shape[0],train_indicator.shape[1]))

In [142]:
idmodel.evaluate([train_latent, train_indicator], 
          LLoutput, batch_size=1505)

1505/1505 [==============================] - 1s 386us/step


93189.4921875

In [143]:
idmodel.layers[1].get_weights()

[array([[ 0.40061963],
        [ 0.43808842],
        [ 0.44014454],
        [-0.21947742],
        [-0.28472468],
        [-0.10219896],
        [-0.45304728],
        [ 0.10144711],
        [ 0.23490828],
        [ 0.21629882],
        [-0.50680506],
        [-0.5622902 ],
        [-0.15840268]], dtype=float32), array([0.], dtype=float32)]

In [144]:
idmodel.layers[3].get_weights()

[array([[-0.78106916, -0.13533974, -0.49468684, -0.29445124, -0.02266002,
         -0.5619515 ]], dtype=float32),
 array([0., 0., 0., 0., 0., 0.], dtype=float32)]

In [145]:
idmodel.layers[7].get_weights()

[array([1., 1., 1., 1., 1., 1., 1.], dtype=float32)]

In [146]:
idmodel.fit([train_latent, train_indicator], 
         LLoutput, epochs=16000, batch_size = 1505)

Epoch 1/16000
1505/1505 [==============================] - 1s 680us/step - loss: 93189.4922
Epoch 2/16000
1505/1505 [==============================] - 0s 9us/step - loss: 90108.7344
Epoch 3/16000
1505/1505 [==============================] - 0s 6us/step - loss: 86785.7812
Epoch 4/16000
1505/1505 [==============================] - 0s 7us/step - loss: 83523.5312
Epoch 5/16000
1505/1505 [==============================] - 0s 8us/step - loss: 80422.9219
Epoch 6/16000
1505/1505 [==============================] - 0s 7us/step - loss: 77523.2344
Epoch 7/16000
1505/1505 [==============================] - 0s 6us/step - loss: 74837.8594
Epoch 8/16000
1505/1505 [==============================] - 0s 5us/step - loss: 72367.2969
Epoch 9/16000
1505/1505 [==============================] - 0s 7us/step - loss: 70104.9219
Epoch 10/16000
1505/1505 [==============================] - 0s 6us/step - loss: 68039.9375
Epoch 11/16000
1505/1505 [==============================] - 0s 5us/step - loss: 66159.1875
Epoch 

Epoch 92/16000
1505/1505 [==============================] - 0s 5us/step - loss: 27421.5781
Epoch 93/16000
1505/1505 [==============================] - 0s 6us/step - loss: 27288.2559
Epoch 94/16000
1505/1505 [==============================] - 0s 7us/step - loss: 27158.6797
Epoch 95/16000
1505/1505 [==============================] - 0s 5us/step - loss: 27032.6758
Epoch 96/16000
1505/1505 [==============================] - 0s 5us/step - loss: 26910.0938
Epoch 97/16000
1505/1505 [==============================] - 0s 5us/step - loss: 26790.7832
Epoch 98/16000
1505/1505 [==============================] - 0s 5us/step - loss: 26674.6211
Epoch 99/16000
1505/1505 [==============================] - 0s 11us/step - loss: 26561.4980
Epoch 100/16000
1505/1505 [==============================] - 0s 6us/step - loss: 26451.3125
Epoch 101/16000
1505/1505 [==============================] - 0s 5us/step - loss: 26343.9785
Epoch 102/16000
1505/1505 [==============================] - 0s 5us/step - loss: 26239.

1505/1505 [==============================] - 0s 6us/step - loss: 22287.8301
Epoch 182/16000
1505/1505 [==============================] - 0s 5us/step - loss: 22260.8027
Epoch 183/16000
1505/1505 [==============================] - 0s 4us/step - loss: 22234.0312
Epoch 184/16000
1505/1505 [==============================] - 0s 6us/step - loss: 22207.5156
Epoch 185/16000
1505/1505 [==============================] - 0s 6us/step - loss: 22181.2422
Epoch 186/16000
1505/1505 [==============================] - 0s 5us/step - loss: 22155.2227
Epoch 187/16000
1505/1505 [==============================] - 0s 5us/step - loss: 22129.4414
Epoch 188/16000
1505/1505 [==============================] - 0s 7us/step - loss: 22103.8984
Epoch 189/16000
1505/1505 [==============================] - 0s 4us/step - loss: 22078.5898
Epoch 190/16000
1505/1505 [==============================] - 0s 6us/step - loss: 22053.5156
Epoch 191/16000
1505/1505 [==============================] - 0s 6us/step - loss: 22028.6680
Epoc

Epoch 271/16000
1505/1505 [==============================] - 0s 5us/step - loss: 20598.2363
Epoch 272/16000
1505/1505 [==============================] - 0s 5us/step - loss: 20585.7305
Epoch 273/16000
1505/1505 [==============================] - 0s 7us/step - loss: 20573.3301
Epoch 274/16000
1505/1505 [==============================] - 0s 7us/step - loss: 20561.0312
Epoch 275/16000
1505/1505 [==============================] - 0s 4us/step - loss: 20548.8359
Epoch 276/16000
1505/1505 [==============================] - 0s 10us/step - loss: 20536.7402
Epoch 277/16000
1505/1505 [==============================] - 0s 5us/step - loss: 20524.7480
Epoch 278/16000
1505/1505 [==============================] - 0s 5us/step - loss: 20512.8555
Epoch 279/16000
1505/1505 [==============================] - 0s 5us/step - loss: 20501.0586
Epoch 280/16000
1505/1505 [==============================] - 0s 5us/step - loss: 20489.3633
Epoch 281/16000
1505/1505 [==============================] - 0s 5us/step - loss

1505/1505 [==============================] - 0s 7us/step - loss: 19807.7441
Epoch 361/16000
1505/1505 [==============================] - 0s 5us/step - loss: 19801.6797
Epoch 362/16000
1505/1505 [==============================] - 0s 6us/step - loss: 19795.6602
Epoch 363/16000
1505/1505 [==============================] - 0s 5us/step - loss: 19789.6855
Epoch 364/16000
1505/1505 [==============================] - 0s 5us/step - loss: 19783.7559
Epoch 365/16000
1505/1505 [==============================] - 0s 4us/step - loss: 19777.8691
Epoch 366/16000
1505/1505 [==============================] - 0s 7us/step - loss: 19772.0254
Epoch 367/16000
1505/1505 [==============================] - 0s 5us/step - loss: 19766.2246
Epoch 368/16000
1505/1505 [==============================] - 0s 5us/step - loss: 19760.4688
Epoch 369/16000
1505/1505 [==============================] - 0s 5us/step - loss: 19754.7500
Epoch 370/16000
1505/1505 [==============================] - 0s 6us/step - loss: 19749.0762
Epoc

...

1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15608/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15609/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9980
Epoch 15610/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15611/16000
1505/1505 [==============================] - 0s 6us/step - loss: 16990.9980
Epoch 15612/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15613/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9961
Epoch 15614/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15615/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15616/16000
1505/1505 [==============================] - 0s 3us/step - loss: 16990.9961
Epoch 15617/16000
1505/1505 [==============================] - 0s 5us/step - l

Epoch 15695/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15696/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9961
Epoch 15697/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15698/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15699/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15700/16000
1505/1505 [==============================] - 0s 3us/step - loss: 16990.9980
Epoch 15701/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15702/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15703/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9980
Epoch 15704/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15705/16000
1505/1505 [==============================]

1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15783/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15784/16000
1505/1505 [==============================] - 0s 3us/step - loss: 16990.9980
Epoch 15785/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16991.0000
Epoch 15786/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16991.0000
Epoch 15787/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16991.0039
Epoch 15788/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16991.0078
Epoch 15789/16000
1505/1505 [==============================] - 0s 6us/step - loss: 16991.0156
Epoch 15790/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16991.0273
Epoch 15791/16000
1505/1505 [==============================] - 0s 6us/step - loss: 16991.0469
Epoch 15792/16000
1505/1505 [==============================] - 0s 3us/step - l

Epoch 15870/16000
1505/1505 [==============================] - 0s 6us/step - loss: 16991.0000
Epoch 15871/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9980
Epoch 15872/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15873/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15874/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16991.0000
Epoch 15875/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15876/16000
1505/1505 [==============================] - 0s 6us/step - loss: 16990.9980
Epoch 15877/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9961
Epoch 15878/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15879/16000
1505/1505 [==============================] - 0s 6us/step - loss: 16990.9980
Epoch 15880/16000
1505/1505 [==============================]

1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15958/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15959/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15960/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15961/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15962/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15963/16000
1505/1505 [==============================] - 0s 3us/step - loss: 16990.9961
Epoch 15964/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9980
Epoch 15965/16000
1505/1505 [==============================] - 0s 5us/step - loss: 16990.9961
Epoch 15966/16000
1505/1505 [==============================] - 0s 4us/step - loss: 16990.9961
Epoch 15967/16000
1505/1505 [==============================] - 0s 6us/step - l

In [38]:
i = 0
for layer in idmodel.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
indicator_ref 2
indicator_other 3
concatenate_3 4
input_y_ture 5
subtract_3 6
ll_3 7


In [134]:
idmodel.layers[1].get_weights()

[array([[-0.12604979],
        [-0.09637873],
        [ 0.36852905],
        [-0.34952155],
        [ 0.62621975],
        [-0.06602555],
        [ 0.0583746 ],
        [ 0.62240404],
        [ 0.21809438],
        [ 0.09953438],
        [-0.04255909],
        [ 0.27948722],
        [-0.6183515 ]], dtype=float32), array([2.5607336], dtype=float32)]

In [117]:
idmodel.layers[2].get_weights()

[array([[1.]], dtype=float32), array([0.], dtype=float32)]

In [118]:
idmodel.layers[3].get_weights()

[array([[-1.7777505,  3.0426996,  2.929085 ,  2.7803855,  2.5307436,
          2.5201962]], dtype=float32),
 array([ 8.0624075, -5.1512814, -4.034674 , -4.1855865, -3.284067 ,
        -3.0818758], dtype=float32)]

In [119]:
idmodel.layers[7].get_weights()

[array([1.4768819, 1.170006 , 1.202057 , 1.1291724, 1.1895471, 1.150193 ,
        1.3095558], dtype=float32)]

计算效用中公共系数部分

In [7]:
class util_common(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(util_common, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_cost_hwh = self.add_weight(name='beta_cost_hwh',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.65),
                                      # constraints = min_max_norm(min_value=-1.650001, max_value=-1.649999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cost_other = self.add_weight(name='beta_cost_other',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.59),
                                      # constraints = min_max_norm(min_value=-1.590001, max_value=-1.589999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(util_common, self).build(input_shape)

    def call(self, inputs):
        hwh = tf.multiply(tf.multiply(inputs[:,0], inputs[:,1]), self.beta_cost_hwh)
        other = tf.multiply(tf.multiply(inputs[:,0], inputs[:,2]), self.beta_cost_other)
        util=tf.transpose(tf.add(hwh, other))
        return tf.reshape(tf.tile(util,[1,random_num]),[-1,random_num])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(util_common, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


V0 alternative specific part

In [8]:
# input [carlover, time_PT_scaled, waitingtimePT]
class V0_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V0_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_time_PT = self.add_weight(name='beta_time_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.02),
                                      # constraints = min_max_norm(min_value=-1.020001, max_value=-1.019999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_PT = self.add_weight(name='beta_cl_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.93),
                                      # constraints = min_max_norm(min_value=1.929999, max_value=1.930001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_waiting_time = self.add_weight(name='self.beta_waiting_time',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.075),
                                      # constraints = min_max_norm(min_value=-0.075001, max_value=-0.074999, rate=1.0, axis=0),           
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V0_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_PT)
        timePT = tf.transpose(tf.multiply(self.beta_time_PT, inputs[1][:,0]))
        waitingtime = tf.transpose(tf.multiply(inputs[1][:,1], self.beta_waiting_time))
        # return tf.transpose(tf.add(timePT, waitingtime))
        return tf.add(timePT, tf.add(waitingtime, cl))
        # return [timePT, waitingtime]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V0_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V1 alternative specific part

In [9]:
# input [carlover, time_CAR_scaled]
class V1_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V1_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.asc_CAR = self.add_weight(name='asc_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=0.741),
                                      # constraints = min_max_norm(min_value=0.740999, max_value=0.741001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_CAR = self.add_weight(name='beta_cl_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-20.4),
                                      # constraints = min_max_norm(min_value=-20.4001, max_value=-20.3999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_time_CAR = self.add_weight(name='beta_time_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=0.483),
                                      # constraints = min_max_norm(min_value=0.482999, max_value=0.483001, rate=1.0, axis=0),          
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V1_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_CAR)
        
        timeCAR = tf.multiply(inputs[1], self.beta_time_CAR)
        
        # return tf.transpose(tf.add(timeCAR, self.asc_CAR))
        return tf.add(tf.add(cl,timeCAR), self.asc_CAR)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V1_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V2 alternative specific part

In [10]:
class V2_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V2_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_dist = self.add_weight(name='beta_dist',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-4.88),
                                      # constraints = min_max_norm(min_value=-4.88001, max_value=-4.87999, rate=1.0, axis=0),   
                                      trainable=trainable)
        self.asc_SM = self.add_weight(name='asc_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.24),
                                      # constraints = min_max_norm(min_value=1.23999, max_value=1.24001, rate=1.0, axis=0), 
                                      trainable=trainable)
        
        # Be sure to call this at the end
        super(V2_as, self).build(input_shape)

    def call(self, inputs):
        return tf.add(tf.multiply(inputs, self.beta_dist), self.asc_SM)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V2_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

Model Specification

In [11]:
### input layers
# batch_size = 32

input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
input_PTcom = keras.Input(shape = (train_PTcom.shape[1],), name = 'input_PTcom')
input_CARcom = keras.Input(shape = (train_CARcom.shape[1],), name = 'input_CARcom')
input_PTasc = keras.Input(shape = (train_PTasc.shape[1],), name = 'input_PTasc')
input_CARasc = keras.Input(shape = (train_CARasc.shape[1],), name = 'input_CARasc')
input_SMasc = keras.Input(shape = (train_SMasc.shape[1],), name = 'input_SMasc')

### car lover
idmodel.layers[1].trainable = False
car_sys = idmodel.layers[1](input_latent)
carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((1,random_num)), name = 'carlover')(car_sys)
# carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((batch_size,20)), name = 'carlover')(car_sys)
# carlover = keras.layers.GaussianNoise(1)(car_sys)

### common coefficients part in utility
utilcommon = util_common(1, name = 'util_common')
PTcom = utilcommon(input_PTcom)
CARcom = utilcommon(input_CARcom)


### alternative specific part in utility
# V0
# util_input0 = keras.layers.concatenate([carlover, input_PTasc], axis = -1)
util_V0as = V0_as(1)([carlover, input_PTasc])
V0 = keras.layers.Add(name = 'V0')([util_V0as, PTcom])

# V1
# util_input1 = keras.layers.concatenate([carlover, input_CARasc], axis = -1)
util_V1as = V1_as(1)([carlover, input_CARasc])
V1 = keras.layers.Add(name = 'V1')([util_V1as, CARcom])

# V2
_V2 = V2_as(1)(input_SMasc)
V2 =keras.layers.Lambda(lambda x: x + tf.zeros((1,random_num)), name = 'V2')(_V2)
### merge
# merged_vector=keras.layers.concatenate([[V0], [V1], [V2]], axis=-1)

prediction =keras.layers.Lambda(lambda x: tf.transpose(tf.reduce_mean(tf.nn.softmax(x, axis = 0),axis=2)), 
                                name = 'prediction')([V0, V1, V2])

model=tf.keras.Model(inputs = [input_latent, input_PTcom, input_CARcom, input_PTasc, input_CARasc, input_SMasc],
                    outputs = prediction)


model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)


In [32]:
util_V0as

<tf.Tensor 'v0_as_3/Add_1:0' shape=(?, 500) dtype=float32>

In [33]:
V0

<tf.Tensor 'V0_3/add:0' shape=(?, 500) dtype=float32>

In [34]:
util_V1as

<tf.Tensor 'v1_as_3/Add_1:0' shape=(?, 500) dtype=float32>

In [13]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y)

1505/1505 [==============================] - 1s 538us/step


[0.6608869326124556, 0.7308970099667774]

In [12]:
#tf.keras.backend.get_value(util_V1as)
callbacks = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, epochs=1000, batch_size = 100)

Epoch 1/1000
1505/1505 [==============================] - 1s 722us/step - loss: 3.7237 - acc: 0.2684
Epoch 2/1000
1505/1505 [==============================] - 0s 190us/step - loss: 3.7607 - acc: 0.2684
Epoch 3/1000
1505/1505 [==============================] - 0s 194us/step - loss: 3.5650 - acc: 0.2704
Epoch 4/1000
1505/1505 [==============================] - 0s 217us/step - loss: 3.3619 - acc: 0.2724
Epoch 5/1000
1505/1505 [==============================] - 0s 199us/step - loss: 3.3594 - acc: 0.2751
Epoch 6/1000
1505/1505 [==============================] - 0s 187us/step - loss: 3.2814 - acc: 0.2804
Epoch 7/1000
1505/1505 [==============================] - 0s 211us/step - loss: 3.0088 - acc: 0.2824
Epoch 8/1000
1505/1505 [==============================] - 0s 181us/step - loss: 2.8851 - acc: 0.2857
Epoch 9/1000
1505/1505 [==============================] - 0s 207us/step - loss: 3.0324 - acc: 0.2983
Epoch 10/1000
1505/1505 [==============================] - 0s 210us/step - loss: 2.8536 - a

1505/1505 [==============================] - 0s 73us/step - loss: 0.8057 - acc: 0.6831
Epoch 162/1000
1505/1505 [==============================] - 0s 81us/step - loss: 0.8046 - acc: 0.6844
Epoch 163/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.8057 - acc: 0.6870
Epoch 164/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.8083 - acc: 0.6850
Epoch 165/1000
1505/1505 [==============================] - 0s 62us/step - loss: 0.8080 - acc: 0.6970
Epoch 166/1000
1505/1505 [==============================] - 0s 75us/step - loss: 0.8023 - acc: 0.6877
Epoch 167/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.8057 - acc: 0.6831
Epoch 168/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.8034 - acc: 0.6897
Epoch 169/1000
1505/1505 [==============================] - 0s 69us/step - loss: 0.8048 - acc: 0.6884
Epoch 170/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.8008 - acc: 0.

1505/1505 [==============================] - 0s 73us/step - loss: 0.7382 - acc: 0.6837
Epoch 322/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7419 - acc: 0.6950
Epoch 323/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7389 - acc: 0.6950
Epoch 324/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7380 - acc: 0.6930
Epoch 325/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.7395 - acc: 0.6924
Epoch 326/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7420 - acc: 0.6963
Epoch 327/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.7387 - acc: 0.6950
Epoch 328/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7396 - acc: 0.6877
Epoch 329/1000
1505/1505 [==============================] - 0s 62us/step - loss: 0.7382 - acc: 0.6937
Epoch 330/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7354 - acc: 0.

1505/1505 [==============================] - 0s 71us/step - loss: 0.7103 - acc: 0.7103
Epoch 482/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7082 - acc: 0.7143
Epoch 483/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7109 - acc: 0.7143
Epoch 484/1000
1505/1505 [==============================] - 0s 65us/step - loss: 0.7100 - acc: 0.7156
Epoch 485/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7109 - acc: 0.7163
Epoch 486/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7106 - acc: 0.7136
Epoch 487/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.7089 - acc: 0.7156
Epoch 488/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.7084 - acc: 0.7150
Epoch 489/1000
1505/1505 [==============================] - 0s 74us/step - loss: 0.7073 - acc: 0.7183
Epoch 490/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.7083 - acc: 0.

1505/1505 [==============================] - 0s 73us/step - loss: 0.6740 - acc: 0.7243
Epoch 642/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6681 - acc: 0.7269
Epoch 643/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.6722 - acc: 0.7243
Epoch 644/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6704 - acc: 0.7256
Epoch 645/1000
1505/1505 [==============================] - 0s 70us/step - loss: 0.6692 - acc: 0.7236
Epoch 646/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6674 - acc: 0.7229
Epoch 647/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.6685 - acc: 0.7256
Epoch 648/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6867 - acc: 0.7136
Epoch 649/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6676 - acc: 0.7276
Epoch 650/1000
1505/1505 [==============================] - 0s 62us/step - loss: 0.6713 - acc: 0.

1505/1505 [==============================] - 0s 73us/step - loss: 0.6646 - acc: 0.7342
Epoch 802/1000
1505/1505 [==============================] - 0s 67us/step - loss: 0.6665 - acc: 0.7302
Epoch 803/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6637 - acc: 0.7289
Epoch 804/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6635 - acc: 0.7355
Epoch 805/1000
1505/1505 [==============================] - 0s 62us/step - loss: 0.6675 - acc: 0.7229
Epoch 806/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6642 - acc: 0.7329
Epoch 807/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.6613 - acc: 0.7329
Epoch 808/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6624 - acc: 0.7382
Epoch 809/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6640 - acc: 0.7309
Epoch 810/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.6642 - acc: 0.

1505/1505 [==============================] - 0s 73us/step - loss: 0.6632 - acc: 0.7342
Epoch 962/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6606 - acc: 0.7342
Epoch 963/1000
1505/1505 [==============================] - 0s 62us/step - loss: 0.6628 - acc: 0.7336
Epoch 964/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6624 - acc: 0.7302
Epoch 965/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6591 - acc: 0.7316
Epoch 966/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.7051 - acc: 0.7236
Epoch 967/1000
1505/1505 [==============================] - 0s 72us/step - loss: 0.6614 - acc: 0.7322
Epoch 968/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6591 - acc: 0.7362
Epoch 969/1000
1505/1505 [==============================] - 0s 71us/step - loss: 0.6646 - acc: 0.7302
Epoch 970/1000
1505/1505 [==============================] - 0s 73us/step - loss: 0.6623 - acc: 0.

In [58]:
model.predict([train_latent[:1], train_PTcom[:1], train_CARcom[:1], train_PTasc[:1], train_CARasc[:1], train_SMasc[:1]])

array([[5.6687373e-01, 4.3312630e-01, 8.0049547e-13]], dtype=float32)

In [39]:
from tensorflow.keras.utils import plot_model
plot_model(idmodel, to_file='linearmodel.png',show_shapes=True)

# Debug part

In [14]:
i = 0
for layer in model.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
carlover 2
input_PTasc 3
input_PTcom 4
input_CARasc 5
input_CARcom 6
input_SMasc 7
v0_as 8
util_common 9
v1_as 10
v2_as 11
V0 12
V1 13
V2 14
prediction 15


In [18]:
model.layers[9].get_weights()

[array([[-1.2084553]], dtype=float32), array([[-0.57047397]], dtype=float32)]

## LatentPart

In [57]:
input_list = [train_latent[:4], train_PTcom[:4], train_CARcom[:4], train_PTasc[:4], train_CARasc[:4], train_SMasc[:4]]
fun_latent = K.function([model.layers[0].input, model.layers[4].input, model.layers[6].input,
                         model.layers[3].input, model.layers[5].input, model.layers[7].input,],
                        [model.layers[1].output, model.layers[2].output,model.layers[8].output, model.layers[9].output, 
                         model.layers[10].output, model.layers[11].output, model.layers[12].output,model.layers[13].output,
                         model.layers[14].output,
                         model.layers[15].output])

output_index = [1,2,8,9,10,11,12,13,14,15]
layer_output = fun_latent(input_list)
for i in range(len(output_index)):
    print('{0:15s} {1}'.format(model.layers[output_index[i]].name, layer_output[i]))
    # print(model.layers[output_index[i]].name, )
# print(layer_output)

car_sys         [[0.16819996]
 [0.77425003]
 [1.8326    ]
 [1.6916    ]]
carlover        [[ 0.3112027   1.0333481   0.08506566  0.450561    0.17618899 -0.24868345
  -0.54998165 -0.8955553   0.59447247  0.39055794  0.79826516  0.5899323
   0.210243   -0.15532458 -1.0824442   0.21003127  0.23885182 -0.45330262
   1.58055    -0.5019848 ]
 [ 0.9172528   1.6393982   0.69111574  1.0566111   0.7822391   0.35736662
   0.05606842 -0.28950524  1.2005225   0.996608    1.4043152   1.1959823
   0.81629306  0.4507255  -0.47639406  0.81608135  0.8449019   0.15274745
   2.1866      0.1040653 ]
 [ 1.9756027   2.6977482   1.7494657   2.1149611   1.840589    1.4157166
   1.1144184   0.7688447   2.2588725   2.0549579   2.462665    2.2543323
   1.8746431   1.5090754   0.5819559   1.8744314   1.9032519   1.2110975
   3.2449498   1.1624153 ]
 [ 1.8346027   2.5567482   1.6084657   1.973961    1.699589    1.2747166
   0.97341835  0.6278447   2.1178725   1.913958    2.3216653   2.1133323
   1.733643    1.368075

In [20]:
for i in range(len(input_list)):
    print(input_list[i])

   ContIncome_0_4000  ContIncome_10000_more  ContIncome_4000_6000  \
0               4.00                      0                     2   
1               3.25                      0                     0   

   ContIncome_6000_8000  ContIncome_8000_10000  age_65_more  haveChildren  \
0                     1                      0            0             0   
1                     0                      0            0             0   

   haveGA  highEducation  individualHouse  male  moreThanOneBike  \
0       1              1                1     1                1   
1       0              0                1     0                0   

   moreThanOneCar  
0               0  
1               0  
   MarginalCostPT_scaled  PurpHWH  PurpOther
0                    0.0        1          0
1                    2.4        1          0
   CostCarCHF_scaled  PurpHWH  PurpOther
0              0.454        1          0
1              0.338        1          0
   TimePT_scaled  WaitingTimePT
0    

In [112]:
for i in range(len(output_index)):
    print('{0:15s} {1}'.format(model.layers[output_index[i]].name, model.layers[output_index[i]].get_weights()))

car_sys         [array([[ 0.12458432],
       [ 0.02909193],
       [ 0.29693022],
       [-0.15076648],
       [ 0.04201582],
       [-0.15310666],
       [-0.07123151],
       [-1.4627142 ],
       [ 0.37573442],
       [ 0.01734441],
       [ 0.1563822 ],
       [-0.04465207],
       [ 0.40894923]], dtype=float32)]
carlover        []
v0_as_9         [array([[-1.6017511]], dtype=float32), array([[2.0203588]], dtype=float32), array([[-0.07922111]], dtype=float32)]
util_common     [array([[-1.675422]], dtype=float32), array([[-1.4764557]], dtype=float32)]
v1_as_9         [array([[0.7524016]], dtype=float32), array([[-21.16343]], dtype=float32), array([[0.5550334]], dtype=float32)]
v2_as_9         [array([[-5.295869]], dtype=float32), array([[1.3265139]], dtype=float32)]
V0              []
V1              []
V2              []
prediction      []


In [37]:
model.layers[9].get_weights()

[array([[-0.2721386]], dtype=float32),
 array([[0.24717349]], dtype=float32),
 array([[0.14008111]], dtype=float32),
 array([[-0.04686852]], dtype=float32)]